In [1]:
import torch
from torch import nn
from tqdm import tqdm
import toml
os.chdir("..")

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from core.dl_framework.data import Dataset
from core.dl_framework.utils import read_config
config_file = read_config(toml.load("configs/default_train_config.toml"))

In [104]:
from core.dl_framework.data import data_pipeline
from core.dl_framework.learner import Learner
from pathlib import Path
import numpy as np
from torchvision import transforms
from sklearn.model_selection import train_test_split
config_file = read_config(toml.load("configs/default_train_config.toml"))
train_data_path = "seg_train/seg_train"
data_test = data_pipeline(config_file, train_data_path, "train_data", True)
learn = Learner(data_test, config_file)


In [ ]:
learn.fit(2)

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [120]:
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)

def conv_block(img_size, n_in, nh, kernel_size, stride, padding, max_kernel=2):
    modules = [
    nn.Conv2d(n_in, nh, kernel_size, stride, padding),
    nn.BatchNorm2d(nh),
    nn.ReLU(),
    nn.MaxPool2d(max_kernel)
    ]
    output_size = conv_output_size(img_size, k=kernel_size, p=padding, s=stride)
    output_size = conv_output_size(output_size, k=max_kernel, p=0, s=max_kernel)
    return modules, int(output_size)

class CustomModel(nn.Module):
    def __init__(self, n_in, n_out, nh, img_size, num_blocks, kernel_size=3, stride=1, padding=1):
        super(CustomModel, self).__init__()
        
        modules_list = []

        for n in range(num_blocks):
            modules, output_size = conv_block(img_size, n_in, nh, kernel_size, stride, padding)
            modules_list.extend(modules)
            
            img_size = output_size
            n_in = nh
            nh = 2*nh
        self.conv = nn.Sequential(*modules_list)
        output_size = output_size**2 * n_in

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(output_size, n_out)
        )
    
    def forward(self, x):
        x = self.conv(x)
        out = self.fc(x)
        return out
    
img = next(iter(data_test.train_dl))[0]
net = CustomModel(img.size(1), 6, 16, 124, 4)
img.size(2)


124

In [114]:
img = next(iter(data_test.train_dl))[0]
img.shape

torch.Size([64, 3, 124, 124])

In [126]:
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)
conv_output_size(124,3,1,1)
conv_output_size(124, 2, 0, 2)
62*62*16
# conv_output_size(1984, 1, 2)

61504